In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
import pandas as pd
nltk.download('stopwords')  # Download stopwords data
data = pd.read_csv("ner_dataset.csv",encoding="latin1" )

# New Section

In [ ]:
data.head(10)

In [ ]:
##Preproccessing the data

In [ ]:
data =data.fillna(method ="ffill")
data['Word']
stop_words = stopwords.words('english')

data = data[~data['Word'].str.match(r'^\W+$')]
data = data[~data['Word'].apply(lambda x: any(word.lower() in stop_words for word in x.split()))]
##data['Word'] = data['Word'].str.lower()
# Reset index after dropping rows
data.reset_index(drop=True, inplace=True)





In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [ ]:
data.head()

In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","POS":"labels"}, inplace =True)

In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2, random_state=45)

In [ ]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [ ]:
x_train

In [ ]:
y_train

# Model Training


In [ ]:
from simpletransformers.ner import NERModel,NERArgs

In [ ]:
label = data["labels"].unique().tolist()
label

In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32


In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda=False)


In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

In [ ]:
result


In [ ]:
prediction, model_output = model.predict(["Dr. Magdy the Egyptian is the best at Cairo"])

In [ ]:
prediction